In [1]:
import sys
sys.path.insert(0, './')

import numpy as np
import keras
from SampledDataset import read_sampled_positions_for_trace, load_saliency, load_true_saliency, get_video_ids, get_user_ids, get_users_per_video, split_list_by_percentage, partition_in_train_and_test_without_any_intersection, partition_in_train_and_test_without_video_intersection, partition_in_train_and_test
from TRACK_model import create_TRACK_model
from CVPR18_model import create_CVPR18_model
import MM18_model
from position_only_baseline import create_pos_only_model
from Pos_Only_Baseline_3dLoss import Pos_Only_Class
from CVPR18_original_model import create_CVPR18_orig_Model, auto_regressive_prediction
from Xu_CVPR_18.Read_Dataset import get_videos_train_and_test_from_file
import os
from Utils import cartesian_to_eulerian, eulerian_to_cartesian, get_max_sal_pos, load_dict_from_csv, all_metrics
import argparse
from Saliency_only_baseline import get_most_salient_points_per_video, predict_most_salient_point
from TRACK_AblatSal_model import create_TRACK_AblatSal_model
from TRACK_AblatFuse_model import create_TRACK_AblatFuse_model
from ContentBased_Saliency_baseline import get_most_salient_content_based_points_per_video, predict_most_salient_cb_point

'''
parser = argparse.ArgumentParser(description='Process the input parameters.')
parser.add_argument('--B', action='store', dest='B', help='Capacity.', type=int)
parser.add_argument('--W_max', action='store', dest='W_max', help='Maximum predicion window.', type=int)
parser.add_argument('--start_user', action='store', dest='start_user', help='Index of start user.', type=int)
parser.add_argument('--end_user', action='store', dest='end_user', help='Index of end user plus one.', type=int)
parser.add_argument('--perturb', action='store', dest='perturb', help='.', type=float)
parser.add_argument('--data_rate', action='store', dest='data_rate', help='.', type=float)
parser.add_argument('--perfect', action='store', dest='perfect', help='.', type=bool)
parser.add_argument('--fix_window', action='store', dest='fix_window', help='.', type=bool)
parser.add_argument('--result_dir', action='store', dest='result_dir', help='Output folder.', type=str)
parser.add_argument('--model_name', action='store', dest='model_name', help='.', type=str)
parser.add_argument('--transmit_power', action='store', dest='transmit_power', help='.', type=int)
args = parser.parse_args()
'''

try:
    if args.B is None:
        B = 300
    else:
        B = args.B
    if args.W_max is None:
        W_max = 5
    else:
        W_max = args.W_max
    if args.result_dir is None:
        result_dir = "../sweet_6x3"
    else:
        result_dir = args.result_dir
    if args.perturb is None:
        perturb = 0.0
    else:
        perturb = args.perturb
    if args.perfect is None:
        perfect = False
    else:
        perfect = args.perfect
    if args.fix_window is None:
        fix_window = False
    else:
        fix_window = args.fix_window
    if args.data_rate is None:
        data_rate = 200
    else:
        data_rate = args.data_rate
    if args.model_name is None:
        model_name = "CVPR18"
    else:
        model_name = args.model_name
    if args.transmit_power is None:
        transmit_power = 15
    else:
        transmit_power = args.transmit_power
        
except:
    print("Except")
    B = 500
    W_max = 5
    result_dir = "../sweet_6x3"
    perturb = 0.0
    perfect = False
    fix_window = False
    data_rate = 200
    model_name = "CVPR18"
    transmit_power=15

evaluate_flag = True
gpu_id = ''
dataset_name = "David_MMSys_18"
#model_name = "CVPR18"
m_window = 5
h_window = 25
init_window = 30
exp_folder = "original_dataset_xyz"
provided_videos = True
if model_name == "TRACK":
    use_true_saliency = False
else:
    use_true_saliency = True
if model_name == "MM18":
    m_window = 15

train_flag = None
end_window = None
metric= None
num_of_peaks = None
video_test_chinacom = None

# Buffer window in timesteps
M_WINDOW = m_window
# Forecast window in timesteps (5 timesteps = 1 second) (Used in the network to predict)
H_WINDOW = h_window
# Initial buffer (to avoid stationary part)
if init_window is None:
    INIT_WINDOW = M_WINDOW
else:
    INIT_WINDOW = init_window
# final buffer (to avoid having samples with less outputs)
if end_window is None:
    END_WINDOW = H_WINDOW
else:
    END_WINDOW = end_window
# This variable is used when we use the dataset of the experiment run in the respective paper
# e.g. Xu_CVPR_18 predicts the gaze using the last 5 gaze positions to predict the next 5 gaze positions
EXP_FOLDER = exp_folder

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = gpu_id

assert dataset_name in ['Xu_PAMI_18', 'AVTrack360', 'Xu_CVPR_18', 'Fan_NOSSDAV_17', 'Nguyen_MM_18', 'David_MMSys_18', 'Li_ChinaCom_18']
assert model_name in ['TRACK', 'CVPR18', 'pos_only', 'no_motion', 'most_salient_point', 'true_saliency', 'content_based_saliency', 'CVPR18_orig', 'TRACK_AblatSal', 'TRACK_AblatFuse', 'MM18', 'pos_only_3d_loss']

if model_name in ['true_saliency', 'content_based_saliency']:
    assert int(num_of_peaks) > 0

if metric is None:
    metric = all_metrics['orthodromic']
else:
    assert metric in all_metrics.keys()
    metric = all_metrics[metric]
    
# Fixing random state for reproducibility
np.random.seed(19680801)

EPOCHS = 500

NUM_TILES_WIDTH = 384
NUM_TILES_HEIGHT = 216

NUM_TILES_WIDTH_TRUE_SAL = 256
NUM_TILES_HEIGHT_TRUE_SAL = 256

RATE = 0.2

root_dataset_folder = os.path.join('./', dataset_name)

# If EXP_FOLDER is defined, add "Paper_exp" to the results name and use the folder in EXP_FOLDER as dataset folder
if EXP_FOLDER is None:
    EXP_NAME = '_init_' + str(INIT_WINDOW) + '_in_' + str(M_WINDOW) + '_out_' + str(H_WINDOW) + '_end_' + str(END_WINDOW)
    SAMPLED_DATASET_FOLDER = os.path.join(root_dataset_folder, 'sampled_dataset')
else:
    EXP_NAME = '_Paper_Exp_init_' + str(INIT_WINDOW) + '_in_' + str(M_WINDOW) + '_out_' + str(H_WINDOW) + '_end_' + str(END_WINDOW)
    SAMPLED_DATASET_FOLDER = os.path.join(root_dataset_folder, EXP_FOLDER)

if dataset_name == 'Li_ChinaCom_18':
    EXP_NAME = EXP_NAME + '_' + video_test_chinacom

SALIENCY_FOLDER = os.path.join(root_dataset_folder, 'extract_saliency/saliency')

if model_name == 'MM18':
    TRUE_SALIENCY_FOLDER = os.path.join(root_dataset_folder, 'mm18_true_saliency')
else:
    TRUE_SALIENCY_FOLDER = os.path.join(root_dataset_folder, 'true_saliency')

if model_name == 'TRACK':
    if use_true_saliency:
        RESULTS_FOLDER = os.path.join(root_dataset_folder, 'TRACK/Results_EncDec_3DCoords_TrueSal' + EXP_NAME)
        MODELS_FOLDER = os.path.join(root_dataset_folder, 'TRACK/Models_EncDec_3DCoords_TrueSal' + EXP_NAME)
    else:
        RESULTS_FOLDER = os.path.join(root_dataset_folder, 'TRACK/Results_EncDec_3DCoords_ContSal' + EXP_NAME)
        MODELS_FOLDER = os.path.join(root_dataset_folder, 'TRACK/Models_EncDec_3DCoords_ContSal' + EXP_NAME)
if model_name == 'TRACK_AblatSal':
    if use_true_saliency:
        RESULTS_FOLDER = os.path.join(root_dataset_folder, 'TRACK_AblatSal/Results_EncDec_3DCoords_TrueSal' + EXP_NAME)
        MODELS_FOLDER = os.path.join(root_dataset_folder, 'TRACK_AblatSal/Models_EncDec_3DCoords_TrueSal' + EXP_NAME)
    else:
        RESULTS_FOLDER = os.path.join(root_dataset_folder, 'TRACK_AblatSal/Results_EncDec_3DCoords_ContSal' + EXP_NAME)
        MODELS_FOLDER = os.path.join(root_dataset_folder, 'TRACK_AblatSal/Models_EncDec_3DCoords_ContSal' + EXP_NAME)
if model_name == 'TRACK_AblatFuse':
    if use_true_saliency:
        RESULTS_FOLDER = os.path.join(root_dataset_folder, 'TRACK_AblatFuse/Results_EncDec_3DCoords_TrueSal' + EXP_NAME)
        MODELS_FOLDER = os.path.join(root_dataset_folder, 'TRACK_AblatFuse/Models_EncDec_3DCoords_TrueSal' + EXP_NAME)
    else:
        RESULTS_FOLDER = os.path.join(root_dataset_folder, 'TRACK_AblatFuse/Results_EncDec_3DCoords_ContSal' + EXP_NAME)
        MODELS_FOLDER = os.path.join(root_dataset_folder, 'TRACK_AblatFuse/Models_EncDec_3DCoords_ContSal' + EXP_NAME)
elif model_name == 'CVPR18':
    if use_true_saliency:
        RESULTS_FOLDER = os.path.join(root_dataset_folder, 'CVPR18/Results_EncDec_3DCoords_TrueSal' + EXP_NAME)
        MODELS_FOLDER = os.path.join(root_dataset_folder, 'CVPR18/Models_EncDec_3DCoords_TrueSal' + EXP_NAME)
    else:
        RESULTS_FOLDER = os.path.join(root_dataset_folder, 'CVPR18/Results_EncDec_3DCoords_ContSal' + EXP_NAME)
        MODELS_FOLDER = os.path.join(root_dataset_folder, 'CVPR18/Models_EncDec_3DCoords_ContSal' + EXP_NAME)
elif model_name == 'pos_only':
    RESULTS_FOLDER = os.path.join(root_dataset_folder, 'pos_only/Results_EncDec_eulerian' + EXP_NAME)
    MODELS_FOLDER = os.path.join(root_dataset_folder, 'pos_only/Models_EncDec_eulerian' + EXP_NAME)
elif model_name == 'pos_only_3d_loss':
    RESULTS_FOLDER = os.path.join(root_dataset_folder, 'pos_only_3d_loss/Results_EncDec_eulerian' + EXP_NAME)
    MODELS_FOLDER = os.path.join(root_dataset_folder, 'pos_only_3d_loss/Models_EncDec_eulerian' + EXP_NAME)
elif model_name == 'CVPR18_orig':
    if use_true_saliency:
        RESULTS_FOLDER = os.path.join(root_dataset_folder, 'CVPR18_orig/Results_EncDec_2DNormalized_TrueSal' + EXP_NAME)
        MODELS_FOLDER = os.path.join(root_dataset_folder, 'CVPR18_orig/Models_EncDec_2DNormalized_TrueSal' + EXP_NAME)
    else:
        RESULTS_FOLDER = os.path.join(root_dataset_folder, 'CVPR18_orig/Results_Enc_2DNormalized_ContSal' + EXP_NAME)
        MODELS_FOLDER = os.path.join(root_dataset_folder, 'CVPR18_orig/Models_Enc_2DNormalized_ContSal' + EXP_NAME)
elif model_name == 'MM18':
    if use_true_saliency:
        RESULTS_FOLDER = os.path.join(root_dataset_folder, 'MM18/Results_Seq2One_2DNormalized_TrueSal' + EXP_NAME)
        MODELS_FOLDER = os.path.join(root_dataset_folder, 'MM18/Models_Seq2One_2DNormalized_TrueSal' + EXP_NAME)

PERC_VIDEOS_TRAIN = 0.8
PERC_USERS_TRAIN = 0.5

BATCH_SIZE = 128.0

TRAIN_MODEL = False
EVALUATE_MODEL = False
if train_flag:
    TRAIN_MODEL = True
if evaluate_flag:
    EVALUATE_MODEL = True

videos = get_video_ids(SAMPLED_DATASET_FOLDER)
users = get_user_ids(SAMPLED_DATASET_FOLDER)
users_per_video = get_users_per_video(SAMPLED_DATASET_FOLDER)

if provided_videos:
    if dataset_name == 'Xu_CVPR_18':
        videos_train, videos_test = get_videos_train_and_test_from_file(root_dataset_folder)
        partition = partition_in_train_and_test_without_video_intersection(SAMPLED_DATASET_FOLDER, INIT_WINDOW, END_WINDOW, videos_train, videos_test, users_per_video)
    if dataset_name == 'Xu_PAMI_18':
        # From PAMI_18 paper:
        # For evaluating the performance of offline-DHP, we randomly divided all 76 panoramic sequences of our PVS-HM database into a training set (61 sequences) and a test set (15 sequences).
        # For evaluating the performance of online-DHP [...]. Since the DRL network of offline-DHP was learned over 61 training sequences and used as the initial model of online-DHP, our comparison was conducted on all 15 test sequences of our PVS-HM database.
        videos_test = ['KingKong', 'SpaceWar2', 'StarryPolar', 'Dancing', 'Guitar', 'BTSRun', 'InsideCar', 'RioOlympics', 'SpaceWar', 'CMLauncher2', 'Waterfall', 'Sunset', 'BlueWorld', 'Symphony', 'WaitingForLove']
        videos_train = ['A380', 'AcerEngine', 'AcerPredator', 'AirShow', 'BFG', 'Bicycle', 'Camping', 'CandyCarnival', 'Castle', 'Catwalks', 'CMLauncher', 'CS', 'DanceInTurn', 'DrivingInAlps', 'Egypt', 'F5Fighter', 'Flight', 'GalaxyOnFire', 'Graffiti', 'GTA', 'HondaF1', 'IRobot', 'KasabianLive', 'Lion', 'LoopUniverse', 'Manhattan', 'MC', 'MercedesBenz', 'Motorbike', 'Murder', 'NotBeAloneTonight', 'Orion', 'Parachuting', 'Parasailing', 'Pearl', 'Predator', 'ProjectSoul', 'Rally', 'RingMan', 'Roma', 'Shark', 'Skiing', 'Snowfield', 'SnowRopeway', 'Square', 'StarWars', 'StarWars2', 'Stratosphere', 'StreetFighter', 'Supercar', 'SuperMario64', 'Surfing', 'SurfingArctic', 'TalkingInCar', 'Terminator', 'TheInvisible', 'Village', 'VRBasketball', 'Waterskiing', 'WesternSichuan', 'Yacht']
        partition = partition_in_train_and_test_without_video_intersection(SAMPLED_DATASET_FOLDER, INIT_WINDOW, END_WINDOW, videos_train, videos_test, users_per_video)
    if dataset_name == 'Fan_NOSSDAV_17':
        train_traces = load_dict_from_csv(os.path.join(root_dataset_folder, 'train_set'), ['user', 'video'])
        test_traces = load_dict_from_csv(os.path.join(root_dataset_folder, 'test_set'), ['user', 'video'])
        videos_test = ['coaster', 'drive', 'pacman', 'game', 'diving', 'coaster2', 'sport', 'ride', 'panel', 'landscape']
        #print(train_traces)
        #print(test_traces)
        partition = partition_in_train_and_test(SAMPLED_DATASET_FOLDER, INIT_WINDOW, END_WINDOW, train_traces, test_traces)
    if dataset_name == 'David_MMSys_18':
        train_traces = load_dict_from_csv(os.path.join(root_dataset_folder, 'train_set'), ['user', 'video'])
        test_traces = load_dict_from_csv(os.path.join(root_dataset_folder, 'test_set'), ['user', 'video'])
        #print(train_traces)
        #print(test_traces)
        partition = partition_in_train_and_test(SAMPLED_DATASET_FOLDER, INIT_WINDOW, END_WINDOW, train_traces, test_traces)
        videos_test = ['1_PortoRiverside', '3_PlanEnergyBioLab', '5_Waterpark', '14_Warship', '16_Turtle']
    if dataset_name == 'Nguyen_MM_18':
        videos_train = ['0', '1', '2', '3', '6']
        videos_test = ['4', '5', '7', '8']
        partition = partition_in_train_and_test_without_video_intersection(SAMPLED_DATASET_FOLDER, INIT_WINDOW, END_WINDOW, videos_train, videos_test, users_per_video)
    if dataset_name == 'Li_ChinaCom_18':
        videos_train = []
        for video in videos:
            if video != video_test_chinacom:
                videos_train.append(video)
        videos_test = [video_test_chinacom]
        partition = partition_in_train_and_test_without_video_intersection(SAMPLED_DATASET_FOLDER, INIT_WINDOW, END_WINDOW, videos_train, videos_test, users_per_video)
else:
    videos_train, videos_test = split_list_by_percentage(videos, PERC_VIDEOS_TRAIN)
    users_train, users_test = split_list_by_percentage(users, PERC_USERS_TRAIN)
    # Datasets
    partition = partition_in_train_and_test_without_any_intersection(SAMPLED_DATASET_FOLDER, INIT_WINDOW, END_WINDOW, videos_train, videos_test, users_train, users_test)

# Dictionary that stores the traces per video and user
all_traces = {}
for video in videos:
    all_traces[video] = {}
    for user in users_per_video[video]:
        all_traces[video][user] = read_sampled_positions_for_trace(SAMPLED_DATASET_FOLDER, str(video), str(user))

if model_name == 'MM18':
    MM18_model.create_gt_sal(TRUE_SALIENCY_FOLDER, all_traces)

# Load the saliency only if it's not the position_only baseline
if model_name not in ['pos_only', 'pos_only_3d_loss', 'no_motion', 'true_saliency', 'content_based_saliency']:
    all_saliencies = {}
    if use_true_saliency:
        for video in videos:
            if model_name == 'MM18':
                all_saliencies[video] = MM18_model.get_true_saliency(TRUE_SALIENCY_FOLDER, video)
            else:
                all_saliencies[video] = load_true_saliency(TRUE_SALIENCY_FOLDER, video)
    else:
        for video in videos:
            all_saliencies[video] = load_saliency(SALIENCY_FOLDER, video)

if model_name == 'MM18':
    all_headmaps = {}
    for video in videos:
        all_headmaps[video] = {}
        for user in users_per_video[video]:
            all_headmaps[video][user] = MM18_model.get_headmaps(all_traces[video][user])

if model_name == 'true_saliency':
    most_salient_points_per_video = get_most_salient_points_per_video(videos, TRUE_SALIENCY_FOLDER, k=int(num_of_peaks))
if model_name == 'content_based_saliency':
    most_salient_points_per_video = get_most_salient_content_based_points_per_video(videos, SALIENCY_FOLDER, k=int(num_of_peaks))

def transform_batches_cartesian_to_normalized_eulerian(positions_in_batch):
    positions_in_batch = np.array(positions_in_batch)
    eulerian_batches = [[cartesian_to_eulerian(pos[0], pos[1], pos[2]) for pos in batch] for batch in positions_in_batch]
    eulerian_batches = np.array(eulerian_batches) / np.array([2*np.pi, np.pi])
    return eulerian_batches

def transform_normalized_eulerian_to_cartesian(positions):
    positions = positions * np.array([2*np.pi, np.pi])
    eulerian_samples = [eulerian_to_cartesian(pos[0], pos[1]) for pos in positions]
    return np.array(eulerian_samples)

def generate_arrays(list_IDs, future_window):
    while True:
        encoder_pos_inputs_for_batch = []
        encoder_sal_inputs_for_batch = []
        decoder_pos_inputs_for_batch = []
        decoder_sal_inputs_for_batch = []
        decoder_outputs_for_batch = []
        count = 0
        np.random.shuffle(list_IDs)
        for ID in list_IDs:
            user = ID['user']
            video = ID['video']
            x_i = ID['time-stamp']
            # Load the data
            if model_name not in ['pos_only', 'pos_only_3d_loss', 'MM18']:
                encoder_sal_inputs_for_batch.append(np.expand_dims(all_saliencies[video][x_i-M_WINDOW+1:x_i+1], axis=-1))
                decoder_sal_inputs_for_batch.append(np.expand_dims(all_saliencies[video][x_i+1:x_i+future_window+1], axis=-1))
            if model_name == 'CVPR18_orig':
                encoder_pos_inputs_for_batch.append(all_traces[video][user][x_i-M_WINDOW+1:x_i+1])
                decoder_outputs_for_batch.append(all_traces[video][user][x_i+1:x_i+1+1])
            elif model_name == 'MM18':
                encoder_sal_inputs_for_batch.append(np.concatenate((all_saliencies[video][x_i-M_WINDOW+1:x_i+1], all_headmaps[video][user][x_i-M_WINDOW+1:x_i+1]), axis=1))
                decoder_outputs_for_batch.append(all_headmaps[video][user][x_i+future_window+1])
            else:
                encoder_pos_inputs_for_batch.append(all_traces[video][user][x_i-M_WINDOW:x_i])
                decoder_pos_inputs_for_batch.append(all_traces[video][user][x_i:x_i+1])
                decoder_outputs_for_batch.append(all_traces[video][user][x_i+1:x_i+future_window+1])
            count += 1
            if count == BATCH_SIZE:
                count = 0
                if model_name in ['TRACK', 'TRACK_AblatSal', 'TRACK_AblatFuse']:
                    yield ([np.array(encoder_pos_inputs_for_batch), np.array(encoder_sal_inputs_for_batch), np.array(decoder_pos_inputs_for_batch), np.array(decoder_sal_inputs_for_batch)], np.array(decoder_outputs_for_batch))
                elif model_name == 'CVPR18':
                    yield ([np.array(encoder_pos_inputs_for_batch), np.array(decoder_pos_inputs_for_batch), np.array(decoder_sal_inputs_for_batch)], np.array(decoder_outputs_for_batch))
                elif model_name == 'pos_only':
                    yield ([transform_batches_cartesian_to_normalized_eulerian(encoder_pos_inputs_for_batch), transform_batches_cartesian_to_normalized_eulerian(decoder_pos_inputs_for_batch)], transform_batches_cartesian_to_normalized_eulerian(decoder_outputs_for_batch))
                elif model_name in ['pos_only_3d_loss']:
                    yield ([np.array(encoder_pos_inputs_for_batch), np.array(decoder_pos_inputs_for_batch)] , np.array(decoder_outputs_for_batch))
                elif model_name == 'CVPR18_orig':
                    yield ([transform_batches_cartesian_to_normalized_eulerian(encoder_pos_inputs_for_batch), np.array(decoder_sal_inputs_for_batch)[:, 0, :, :, 0]], transform_batches_cartesian_to_normalized_eulerian(decoder_outputs_for_batch)[:, 0])
                elif model_name == 'MM18':
                    yield (np.array(encoder_sal_inputs_for_batch), np.array(decoder_outputs_for_batch))
                encoder_pos_inputs_for_batch = []
                encoder_sal_inputs_for_batch = []
                decoder_pos_inputs_for_batch = []
                decoder_sal_inputs_for_batch = []
                decoder_outputs_for_batch = []
        if count != 0:
            if model_name == 'TRACK':
                yield ([np.array(encoder_pos_inputs_for_batch), np.array(encoder_sal_inputs_for_batch), np.array(decoder_pos_inputs_for_batch), np.array(decoder_sal_inputs_for_batch)], np.array(decoder_outputs_for_batch))
            elif model_name == 'CVPR18':
                yield ([np.array(encoder_pos_inputs_for_batch), np.array(decoder_pos_inputs_for_batch), np.array(decoder_sal_inputs_for_batch)], np.array(decoder_outputs_for_batch))
            elif model_name == 'pos_only':
                yield ([transform_batches_cartesian_to_normalized_eulerian(encoder_pos_inputs_for_batch), transform_batches_cartesian_to_normalized_eulerian(decoder_pos_inputs_for_batch)], transform_batches_cartesian_to_normalized_eulerian(decoder_outputs_for_batch))
            elif model_name in ['pos_only_3d_loss']:
                yield ([np.array(encoder_pos_inputs_for_batch), np.array(decoder_pos_inputs_for_batch)] , np.array(decoder_outputs_for_batch))
            elif model_name == 'CVPR18_orig':
                yield ([transform_batches_cartesian_to_normalized_eulerian(encoder_pos_inputs_for_batch), np.array(decoder_sal_inputs_for_batch)[:, 0, :, :, 0]], transform_batches_cartesian_to_normalized_eulerian(decoder_outputs_for_batch)[:, 0])
            elif model_name == 'MM18':
                yield (np.array(encoder_sal_inputs_for_batch), np.array(decoder_outputs_for_batch))

if model_name == 'TRACK':
    if use_true_saliency:
        model = create_TRACK_model(M_WINDOW, H_WINDOW, NUM_TILES_HEIGHT_TRUE_SAL, NUM_TILES_WIDTH_TRUE_SAL)
    else:
        model = create_TRACK_model(M_WINDOW, H_WINDOW, NUM_TILES_HEIGHT, NUM_TILES_WIDTH)
elif model_name == 'TRACK_AblatSal':
    if use_true_saliency:
        model = create_TRACK_AblatSal_model(M_WINDOW, H_WINDOW, NUM_TILES_HEIGHT_TRUE_SAL, NUM_TILES_WIDTH_TRUE_SAL)
    else:
        model = create_TRACK_AblatSal_model(M_WINDOW, H_WINDOW, NUM_TILES_HEIGHT, NUM_TILES_WIDTH)
elif model_name == 'TRACK_AblatFuse':
    if use_true_saliency:
        model = create_TRACK_AblatFuse_model(M_WINDOW, H_WINDOW, NUM_TILES_HEIGHT_TRUE_SAL, NUM_TILES_WIDTH_TRUE_SAL)
    else:
        model = create_TRACK_AblatFuse_model(M_WINDOW, H_WINDOW, NUM_TILES_HEIGHT, NUM_TILES_WIDTH)
elif model_name == 'CVPR18':
    if use_true_saliency:
        model = create_CVPR18_model(M_WINDOW, H_WINDOW, NUM_TILES_HEIGHT_TRUE_SAL, NUM_TILES_WIDTH_TRUE_SAL)
    else:
        model = create_CVPR18_model(M_WINDOW, H_WINDOW, NUM_TILES_HEIGHT, NUM_TILES_WIDTH)
elif model_name == 'MM18':
    mm18_models = []
    for _h_window in range(H_WINDOW):
        mm18_models.append(MM18_model.create_MM18_model())
elif model_name == 'pos_only':
    model = create_pos_only_model(M_WINDOW, H_WINDOW)
elif model_name == 'pos_only_3d_loss':
    obj = Pos_Only_Class(H_WINDOW)
    model = obj.get_model()
elif model_name == 'CVPR18_orig':
    if use_true_saliency:
        model = create_CVPR18_orig_Model(M_WINDOW, NUM_TILES_HEIGHT_TRUE_SAL, NUM_TILES_WIDTH_TRUE_SAL)
    else:
        model = create_CVPR18_orig_Model(M_WINDOW, NUM_TILES_HEIGHT, NUM_TILES_WIDTH)

if model_name not in ['no_motion', 'most_salient_point', 'true_saliency', 'content_based_saliency', 'MM18']:
    model.summary()
elif model_name == 'MM18':
    mm18_models[0].summary()

steps_per_ep_train = np.ceil(len(partition['train'])/BATCH_SIZE)
steps_per_ep_validate = np.ceil(len(partition['test'])/BATCH_SIZE)

if TRAIN_MODEL:
    # Create results folder and models folder
    if not os.path.exists(RESULTS_FOLDER):
        os.makedirs(RESULTS_FOLDER)
    if not os.path.exists(MODELS_FOLDER):
        os.makedirs(MODELS_FOLDER)

    if model_name == 'MM18':
        for _h_window in range(H_WINDOW):
            csv_logger = keras.callbacks.CSVLogger(RESULTS_FOLDER + '/results_h_window_' + str(_h_window) + '.csv')
            model_checkpoint = keras.callbacks.ModelCheckpoint(MODELS_FOLDER + '/weights_h_window_' + str(_h_window) + '.hdf5', save_best_only=True, save_weights_only=True, mode='auto', period=1)
            mm18_models[_h_window].fit_generator(generator=generate_arrays(partition['train'], future_window=_h_window), verbose=1, steps_per_epoch=steps_per_ep_train, epochs=EPOCHS, callbacks=[csv_logger, model_checkpoint], validation_data=generate_arrays(partition['test'], future_window=_h_window), validation_steps=steps_per_ep_validate)
    else:
        csv_logger = keras.callbacks.CSVLogger(RESULTS_FOLDER + '/results.csv')
        model_checkpoint = keras.callbacks.ModelCheckpoint(MODELS_FOLDER + '/weights.hdf5', save_best_only=True, save_weights_only=True, mode='auto', period=1)
        model.fit_generator(generator=generate_arrays(partition['train'], future_window=H_WINDOW), verbose=1, steps_per_epoch=steps_per_ep_train, epochs=EPOCHS, callbacks=[csv_logger, model_checkpoint], validation_data=generate_arrays(partition['test'], future_window=H_WINDOW), validation_steps=steps_per_ep_validate)
if EVALUATE_MODEL:
    if model_name not in ['no_motion', 'most_salient_point', 'true_saliency', 'content_based_saliency', 'MM18']:
        model.load_weights(MODELS_FOLDER + '/weights.hdf5')
    elif model_name == 'MM18':
        for _h_window in range(H_WINDOW):
            mm18_models[_h_window].load_weights(MODELS_FOLDER + '/weights_h_window_' + str(_h_window) + '.hdf5')

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

Except



prop_out_enc_1.shape (?, ?, 256)
prop_out_enc_2.shape (?, 256)
inputs.shape (?, 1, 3)
prop_out_dec_1.shape (?, ?, 256)
prop_out_dec_2.shape (?, 256)
prop_out_dec_2_timestep.shape (?, 1, 256)
conc_out_dec.shape (?, 1, 65792)
fuse_out_1_dec.shape (?, 1, 256)
fuse_out_2_dec.shape (?, 1, 256)
decoder_pred.shape (?, 1, 3)
inputs.shape (?, 1, 3)
prop_out_dec_1.shape (?, ?, 256)
prop_out_dec_2.shape (?, 256)
prop_out_dec_2_timestep.shape (?, 1, 256)
conc_out_dec.shape (?, 1, 65792)
fuse_out_1_dec.shape (?, 1, 256)
fuse_out_2_dec.shape (?, 1, 256)
decoder_pred.shape (?, 1, 3)
inputs.shape (?, 1, 3)
prop_out_dec_1.shape (?, ?, 256)
prop_out_dec_2.shape (?, 256)
prop_out_dec_2_timestep.shape (?, 1, 256)
conc_out_dec.shape (?, 1, 65792)
fuse_out_1_dec.shape (?, 1, 256)
fuse_out_2_dec.shape (?, 1, 256)
decoder_pred.shape (?, 1, 3)
inputs.shape (?, 1, 3)
prop_out_dec_1.shape (?, ?, 256)
prop_out_dec_2.shape (?, 256)
prop_out_dec_2_timestep.shape (?, 1, 256)
conc_out_dec.shape (?, 1, 65792

In [2]:
import math
from scipy.optimize import minimize_scalar
from scipy import special
from numpy import linalg, random
import pickle
from itertools import product
import time
import os.path
import urllib

In [3]:
#P = [[.4, .3, .2, .1],
#     [.25, .3, .25, .2],
#     [.2, .25, .3, .25],
#     [.1, .2, .3, .4]]
P = [[0.28, 0.72], [0.19, 0.81]]

def PL_LOS(f, d):
    d = max(d, 1)
    
    c = 299792458
    n =1.73
    #sigma = 3.02

    #print(20 * np.log10(4 * np.pi * f / c))
    #print(10 * n * np.log10(d))
    
    return 20 * np.log10(4 * np.pi * f / c) + 10 * n * np.log10(d)

def PL_NLOS_dual(f, d):
    d = max(d, 1)
    
    c =  299792458
    d_BP = 7.8
    n1 = 2.51
    n2 = 4.25
    b1 = 0.12
    b2 = 0.04
    #sigma = 7.65
    f0 = 24100000000

    if d <= d_BP:
        return 20 * np.log10(4 * np.pi * f / c) + \
               10 * n1 * (1 + b1 * (f - f0) / f0) * np.log10(d)

    return 20 * np.log10(4 * np.pi * f / c) + \
           10 * n1 * (1 + b1 * (f - f0) / f0) * np.log10(d_BP) + \
           10 * n2 * (1 + b2 * (f - f0) / f0) * np.log10(d / d_BP)

def SNR(h, p, BW):
    BW = np.log10(BW) * 10
    p = np.log10(p) * 10
    N_0 = -174
    NF = 9
    
    print(f"p = {p}, h = {h}, BW = {BW}")
    print(f"SNR: {p - h - N_0 - BW - NF} dB")
    #return pow(10, (p - h - N_0 - BW - NF) / 10)
    return p - h - N_0 - BW - NF

def Zeta(e, gamma, sigma, C_min):
    x = 10 / np.log2(10) * C_min / e
    #print(f"e = {e}, x = {x}, mu = {gamma}, zeta = {1 - 0.5 * (1 + special.erf((x - gamma) / sigma / 2**0.5))}")
    return 1 - 0.5 * (1 + special.erf((x - gamma) / sigma / 2**0.5))
    #tmp = e / pow(0.8, 3) / (s+1)
    #return 2 / (1 + np.exp(-2 * tmp)) - 1
     
def determin_C_min(j, data_rate):
    if j == 1 or j == 2:
        return data_rate
    else:
        return data_rate / 2
    
def K(t, d):
    alpha = 1
    return pow(t, 2*alpha/(3*alpha+d)) * np.log10(t)

def xyz_to_eqr(x,y,z):
    if x == 0:
        if y >= 0:
            theta = np.pi/2
        else:
            theta = -np.pi/2
    else:
        theta = np.arctan(y/x)
        if x < 0:
            if theta >= 0:
                theta -= np.pi
            else:
                theta += np.pi
    if pow(x,2)+pow(y,2) == 0:
        if z > 0:
            phi = np.pi/2
        else:
            phi = -np.pi/2
    else:
        phi = np.arctan(z/np.sqrt(pow(x,2)+pow(y,2)))

    #_x = int(np.floor((theta + np.pi) / (2*np.pi) * N_x))
    #_y = int(np.floor((phi + np.pi/2) / (np.pi) * N_y))
    return [theta, phi]

def eqr_to_xyz(theta, phi):
    return np.cos(phi)*np.cos(theta), np.cos(phi)*np.sin(theta), np.sin(phi)

def tile_to_xyz(i, j, N_x, N_y):
    eqr = tile_to_eqr(i, j, N_x, N_y)
    
    x = np.cos(eqr[0]) * np.cos(eqr[1])
    y = np.sin(eqr[0]) * np.cos(eqr[1])
    z = np.sin(eqr[1])
    
    return x,y,z

def tile_to_eqr(i, j, N_x, N_y):
    theta = (-1 + (2*i + 1) / N_x) * np.pi
    phi = (-1 + (2*j + 1) / N_y) * np.pi / 2
    
    return [theta, phi]

def eqr_to_tile(x, y, N_x, N_y):
    x += np.pi
    y += np.pi/2
    
    i = int(np.floor(x / np.pi / 2 * N_x)) % N_x
    j = int(np.floor(y / np.pi * N_y)) % N_y
    
    return [i,j]

def fov_edge(pitch, theta):
    theta = theta / 180 * np.pi
    pitch = pitch / 180 * np.pi
    rotate = np.array([[np.cos(pitch), 0, np.sin(pitch)], \
                      [0, 1, 0], \
                      [-np.sin(pitch), 0, np.cos(pitch)]])
    ref = np.array([np.cos(theta), np.sin(theta), 0])
    corner = np.matmul(rotate, np.transpose(ref))

    lon = np.arccos(corner[0] / np.linalg.norm(corner[:2]))
    if theta != 0:
        lon *= np.abs(theta) / theta
    lat = np.arccos(corner[0] * np.cos(lon) + corner[1] * np.sin(lon))
    if pitch != 0:
        lat *= np.abs(pitch) / pitch

    #return lon / np.pi * 180, lat / np.pi * 180
    return lon, lat

def collect_viewport(offset_x, offset_y, width, height, N_x, N_y):
    _x = offset_x
    offset_x = int(offset_x / np.pi * 180)
    offset_y = int(offset_y / np.pi * 180)
    width = int(width / np.pi * 180)
    height = int(height / np.pi * 180)
    
    collect_tile = set()
    
    for _ in range(2):
        for p in range(-height + offset_y, height + 1 + offset_y, 2):
            tmp = fov_edge(p, width)
            collect_tile.add(eqr_to_tile(tmp[0] + _x, tmp[1], N_x, N_y))
        for p in range(width, -width-1, -2):
            tmp = fov_edge(height + offset_y, p)
            collect_tile.add(eqr_to_tile(tmp[0] + _x, tmp[1], N_x, N_y))
        for p in range(height + offset_y, -height-1 + offset_y, -2):
            tmp = fov_edge(p, -width)
            collect_tile.add(eqr_to_tile(tmp[0] + _x, tmp[1], N_x, N_y))
        for p in range(-width, width+1, 2):
            tmp = fov_edge(-height + offset_y, p)
            collect_tile.add(eqr_to_tile(tmp[0] + _x, tmp[1], N_x, N_y))
        
        width //= 2
        height //= 2
    
    return collect_tile

def ortho_distance(x, y, x_t, y_t):
    return np.arccos(np.cos(x-x_t)*np.cos(y)*np.cos(y_t) + np.sin(y)*np.sin(y_t))

def distortion(y_t):
    return np.cos(y_t)

def compute_QoE(x, y, x_t, y_t):
    return distortion(y_t) / (ortho_distance(x,y,x_t,y_t)+1)

def compute_QoE_dis_lat(d, l):
    return distortion(l) / (d + 1) / 4

def compute_all_QoE(x, y, N_x, N_y, width, height):
    V = collect_viewport(x, y, width, height, N_x, N_y)
    for (i,j) in V:
        eqr = tile_to_eqr(i, j, N_x, N_y)
        compute_QoE(x, y, eqr[0], eqr[1])

#def zeta(s, e):
#    tmp = e / pow(0.8, 3) / (s+1)
#    return 2 / (1 + np.exp(-2 * tmp)) - 1

def value_function(a, s, t, tau, beta, snr_los_nlos, C_min, W, lamb, policy, value, verbose):
    if a == 1:
        value[(a, s, t)] = beta
        return
    else:
        if t == tau:
            value[(a, s, t)] = 0
            return
            #return 0
        
        tmp1 = 0
        tmp2 = 0
        
        for _s in range(len(P)):
            if not (1, _s, t+1) in value:
                value_function(1, _s, t+1, tau, beta, snr_los_nlos, C_min, W, lamb, policy, value, verbose)
            tmp1 += P[s][_s] * value[(1, _s, t+1)]
            
            if not (0, _s, t+1) in value:
                value_function(0, _s, t+1, tau, beta, snr_los_nlos, C_min, W, lamb, policy, value, verbose)
            tmp2 += P[s][_s] * value[(0, _s, t+1)]
        
        #print(f"detail: w = {w}, tau = {tau}, t = {t}")
        #print(lamb)
        
        def value_with_e(e):
            #print(f"e = {e}")
            #print(f"tmp1 = {tmp1}")
            #print(f"tmp2 = {tmp2}")
            zt = Zeta(e, snr_los_nlos[s], shadow_sigma, C_min)
            return -(-lamb[t] * e + zt * tmp1 + (1 - zt) * tmp2)
        #if (x, tau, s, t) == (0, 1, 3, 1):
        #print(f"lamb[{w+1-tau}] = {lamb[w+1-tau]}")
        #print(f"tmp1 = {tmp1}")
        #print(f"tmp2 = {tmp2}")
        #for e in range(0, 30, 5):
        #    print(e, -value_with_e(e))
        
        sol = minimize_scalar(value_with_e, bounds=(1, 70), method='bounded', tol=3)
        #if verbose:
            #if t == 0:
            #    print(f"({x}, {tau}, {s}, {t}), tmp1 = {tmp1}, tmp2 = {tmp2}, diff: {tmp1 - tmp2}")
            #if tau+t == 2:
            #    print(f"        ({x}, {tau}, {s}, {t}): tmp1 = {tmp1}, tmp2 = {tmp2}, diff: {tmp1 - tmp2}")
        
        #if tau+t-1 == 0 and 45 > sol.x > 5:
        #    print(f"({x}, {tau}, {s}, {t}), sol.x: {sol.x}")
        #print(f"sol = {sol}")
        
        #if (x, tau, s, t) == (0, 1, 3, 1):
        #    print(f"tmp1 = {tmp1}")
        #    print(f"tmp2 = {tmp2}")
        #    print(f"e* = {sol.x}")
        #    print(f"value = {sol.fun}")
        
        #print(f"e_star = {e_star}")
        #if lamb[t]>100:
        #    print(f"state = {x}, {tau}, {s}, {t}, lamb[tau-1] = {lamb[tau-1]}")
        #    print(tmp1)
        #    print(tmp2)
        #    print(e_star)
        #    print()
        
        if -sol.fun <= tmp2:
            policy[(a, s, t)] = 0.
            value[(a, s, t)] = tmp2
            return
        
        if sol.x < 5:
            policy[(a, s, t)] = 0.
            value[(a, s, t)] = tmp2
            return
        else:
            policy[(a, s, t)] = sol.x
            value[(a, s, t)] = -sol.fun# + lamb[t] * sol.x
        #print(sol.x)
        #print(f"fun = {sol.fun}")
        #if -sol.fun > 100:
        #    print(f"value = {-sol.fun}")
        #    print(f"policy = {sol.x}")
        #    print(f"tmp1 = {tmp1}")
        #    print(f"tmp2 = {tmp2}")
        return
        #return -sol.fun

In [ ]:
# FoV coverage -> Tile set
def collect_fixed_tile(center, angle, N_x, N_y):
    #center = np.matmul(proj, center)
    O = (np.cos(angle), 0, 0)
    ref = (O[0], np.sin(angle), 0)
    lon, lat = xyz_to_eqr(center[0], center[1], center[2])
    
    samples = []
    
    for i in range(3):
        a = angle / (i+1)
        ref = (np.cos(a), np.sin(a), 0)
        for j in range(48):
            samples.append(np.matmul(R_x(2*np.pi*j/48), ref))
    
    result = np.zeros((N_x, N_y))
    for s in samples:
        s = np.matmul(R_y(-lat), s)
        s = np.matmul(R_z(lon), s)
        theta, phi = xyz_to_eqr(s[0], s[1], s[2])
        i, j = eqr_to_tile(theta, phi, N_x, N_y)
        
        result[i][j] = 1
    
    return result

def R_x(a):
    return np.array([[1, 0, 0],\
                     [0, np.cos(a), -np.sin(a)],\
                     [0, np.sin(a), np.cos(a)]])

def R_y(a):
    return np.array([[np.cos(a), 0, np.sin(a)],\
                     [0, 1, 0],\
                     [-np.sin(a), 0, np.cos(a)]])

def R_z(a):
    return np.array([[np.cos(a), -np.sin(a), 0],\
                     [np.sin(a), np.cos(a), 0],\
                     [0, 0, 1]])

def objective(rates, Q_last, weights, C, Fs, t_miss, buf, N_x, N_y):
    # Q1
    theta = 200000.
    Q1 = np.zeros(len(rates))
    for t in range(len(rates)):
        # q = [11, 20, 44, 64]
        MSE = 0.76 * np.exp((np.log(theta) - np.log(rates[t] * (N_x * N_y) ** 0.9)) / 1.195) + 0.68
        MSE = MSE.flatten()
        #print("log(rate): ", np.log(rates[t]))
        #print("log(theta): ", np.log(theta))
        #print(MSE)
        w = weights[t].flatten()
        w = w[np.isfinite(MSE)]
        MSE = MSE[np.isfinite(MSE)]
        MSE = np.average(MSE, weights=w)
        
        Q1[t] = 10 * np.log10(255**2 / MSE)
    #print("Q1: ", Q1)
    
    # Q2
    Q2 = 0.
    for t in range(len(rates)-1):
        Q2 += abs(Q1[t+1] - Q1[t])
    Q2 += abs(Q1[0] - Q_last)
    #print("Q2: ", Q2)
    # Q3
    Q3 = 0.
    for t in range(len(rates)):
        buf -= np.sum(rates[t])/C[t]
        if buf < 0:
            Q3 += t_miss - buf
            buf = 0
        buf += 1 / Fs
    #print("Q3: ", Q3)
    
    return 0.1 * np.sum(Q1) - 0.2 * Q2 - 0.4 * Q3

In [ ]:
Video = ['1_PortoRiverside', '2_Diner', '3_PlanEnergyBioLab', '4_Ocean', '5_Waterpark', \
         '6_DroneFlight', '7_GazaFishermen', '8_Sofa', '9_MattSwift', '10_Cows', \
         '11_Abbottsford', '12_TeatroRegioTorino', '13_Fountain', '14_Warship', '15_Cockpit', \
         '16_Turtle', '17_UnderwaterPark', '18_Bar', '19_Touvet']
#Video = ['1_PortoRiverside', '2_Diner', '3_PlanEnergyBioLab', '4_Ocean', '5_Waterpark']

try:
    User = list(map(str, range(args.start_user, args.end_user)))
except:
    print("Except2")
    User = list(map(str, range(2, 50)))

#all_Nx_Ny = [[4, 4], [5, 5], [6, 6], [7, 7], [8, 8], [9, 9], [10, 10]]
N_x = 6
N_y = 3
width = np.pi/3
height = np.pi/3
alpha = 0.9
beta = 50
Fs = 5
B_defult = 3
B_max = 10
horizon = 10
past_horizon = 5
t_miss = 1 / Fs
Q_last = 50.

ori_bandwidth_trace = []
for i in range(1, 2):
    url = f"https://users.ugent.be/~jvdrhoof/dataset-4g/logs/report_foot_000{i}.log"
    file = urllib.request.urlopen(url)
    for line in file:
        decoded_line = line.decode("utf-8")
        band, duration = decoded_line.split(" ")[-2:]
        if abs(int(duration)) - 1000 < 10:
            ori_bandwidth_trace.append(int(band))
bandwidth_trace = []
for i in range(len(ori_bandwidth_trace)-1):
    delta = (ori_bandwidth_trace[i+1] - ori_bandwidth_trace[i]) / Fs
    bandwidth_trace.append(ori_bandwidth_trace[i])
    for f in range(1, Fs):
        bandwidth_trace.append(ori_bandwidth_trace[i] + delta * f)
print(bandwidth_trace)
    
for user, video in product(User, Video):
    if not os.path.isdir(result_dir):
        os.mkdir(result_dir)
    '''
    if os.path.isfile(f"{result_dir}/S_{W_max}_{B}_{video}_{user}.pkl"):
        _file = open(f"{result_dir}/S_{W_max}_{B}_{video}_{user}.pkl", "rb")
        data = pickle.load(_file)
        if data[-1] != -1:
            print(f"{result_dir}/S_{W_max}_{B}_{video}_{user}.pkl exists!")
            continue
        else:
            print(f"{result_dir}/S_{W_max}_{B}_{video}_{user}.pkl is not yet completed!")
            print(data)
    '''
    if os.path.isfile(f"{result_dir}/S_{W_max}_{B}_{video}_{user}.pkl"):
        skip_flag = True
        continue
    
    print(user, video)
    print(f"Start to process")
    
    sal_shape = all_saliencies[video].shape
    tmp_all_saliencies = np.zeros((sal_shape[0]+M_WINDOW+H_WINDOW, sal_shape[1], sal_shape[2]))
    for _t in range(M_WINDOW):
        tmp_all_saliencies[_t] = all_saliencies[video][0]
    tmp_all_saliencies[M_WINDOW:sal_shape[0]+M_WINDOW] = all_saliencies[video]
    for _t in range(H_WINDOW):
        tmp_all_saliencies[-_t-1] = all_saliencies[video][-1]
    tra_shape = all_traces[video][user].shape
    tmp_all_traces = np.zeros((tra_shape[0]+M_WINDOW+H_WINDOW, tra_shape[1]))
    for _t in range(M_WINDOW):
        tmp_all_traces[_t] = all_traces[video][user][0]
    tmp_all_traces[M_WINDOW:tra_shape[0]+M_WINDOW] = all_traces[video][user]
    for _t in range(H_WINDOW):
        tmp_all_traces[-_t-1] = all_traces[video][user][-1] 
    
    T = all_traces[video][user].shape[0]
    prediction = {}
    ground = {}
    tolerance = np.zeros((4,))
    error = [0, 0]
    received_rate = {}
    buffer = B_defult
    rate_table = np.arange(400000, 6400000, 400000)
    #rate_table = np.array([400000., 800000., 1600000., 2400000., 3600000., 4800000., 5600000., 6400000.])
    rate_table = rate_table / (N_x * N_y) ** 0.9
    
    Buffer = {}
    Weights = {}
    Bandwidth_trace = {}
    Bandwidth_pred = {}
    Received_rate = {}
    Received_qoe = {}
    
    for i in range(buffer * Fs):
        received_rate[i] = np.ones((N_x, N_y)) * rate_table[-1]
    
    for t in range(T-W_max):
        # Observation from the predictor
        encoder_sal = np.array([np.expand_dims(tmp_all_saliencies[t + 1:t + M_WINDOW + 1], axis=-1)])
        decoder_sal = np.array([np.expand_dims(tmp_all_saliencies[t + M_WINDOW + 1:t + M_WINDOW + H_WINDOW + 1], axis=-1)])
        encoder_pos = np.array([tmp_all_traces[t:t + M_WINDOW]])
        decoder_pos = np.array([tmp_all_traces[t + M_WINDOW:t + M_WINDOW + 1]])
        
        tmp = model.predict([np.array(encoder_pos), np.array(decoder_pos), np.array(decoder_sal)])[0]
        
        tmp = list(map(lambda x: x / np.linalg.norm(x), tmp))
        prediction[t] = tmp
        ground[t] = all_traces[video][user][t]
        lon, lat = xyz_to_eqr(ground[t][0], ground[t][1], ground[t][2])
        
        # Bandwidth prediction to-do
        bandwidth_pred = []
        if t == 0:
            bandwidth_pred = [bandwidth_trace[0]] * horizon
        else:
            for h in range(horizon):
                if h == 0:
                    bandwidth_pred.append(np.average(bandwidth_trace[max(0, t-past_horizon) : t]))
                else:
                    bandwidth_pred.append((np.sum(bandwidth_trace[min(max(0, t-past_horizon+h), t) : t])\
                                          + np.sum(bandwidth_pred)) / past_horizon)
        print("bandwidth_pred: ", bandwidth_pred)
        Bandwidth_trace[t] = bandwidth_trace[t]
        Bandwidth_pred[t] = bandwidth_pred
        
        if buffer > B_max:
            buffer -= 1 / Fs
            continue
        
        w1 = {}
        w2 = {}
        weights = {}
        rate_table = np.arange(800000, 6400000, 800000)
        #rate_table = np.array([400000., 800000., 1600000., 2400000., 3600000., 4800000., 5600000., 6400000.])
        rate_table = rate_table / (N_x * N_y) ** 0.9
        
        for h in range(horizon):
            w1[h] = collect_fixed_tile(prediction[t][h], np.pi / 180 * 60, N_x, N_y)
            w2[h] = collect_fixed_tile(prediction[t][h], np.pi / 180 * 25, N_x, N_y)
            weights[h] = w1[h] + w2[h]
        Weights[t] = weights[0]
        
        record_rate = {}
        record_qoe = {}
        count = 0
        for i in range(len(rate_table)):
            for j in range(min(len(rate_table), i+1)):
        #for j in range(len(rate_table)):
                rates = np.zeros((horizon, N_x, N_y))
                for h in range(horizon):
                    rates[h] = w1[h] * rate_table[j] + w2[h] * (rate_table[i] - rate_table[j])
                
                #if rates[0][0][0] == 200:
                    #print("check: ", rates[0])
                    #print("check: ", w1[0], w2[0])
                record_qoe[count] = objective(rates, Q_last, weights, bandwidth_pred, Fs, t_miss, buffer, N_x, N_y)
                record_rate[count] = rates
                count += 1

        #print(len(record))
        received_qoe = max(record_qoe.values())
        best = max(record_qoe, key=record_qoe.get)
        received_rate = record_rate[best]
        
        #print("w1: ", w1)
        #print("w2: ", w2)
        #print("rates: ", rates)
        #print("received_rate: ", received_rate[0])
        
        buffer += 1/Fs - np.sum(received_rate[0]) / bandwidth_trace[t]
        
        print(record_qoe)
        #print(record_rate)
        print("ground band: ", bandwidth_trace[t])
        print("pred band: ", bandwidth_pred[0])
        
        print("buffer: ", buffer)
        
        Buffer[t] = buffer
        Received_rate[t] = received_rate[0]
        Received_qoe[t] = received_qoe
        
        # to-do
        
        ## bandwidth prediction
        # load the dataset
        # write the bandwidth predictor using python module
        # obtain bandwidth_pred when needed
        
        ## QoE
        # complete rates by w1 w2
        # verify the input and output for QoE
        # complete the solver to efficiently loop over the solution space
        # 
        
        ## plot and verify
        # select the parameter to save
        # plot them up
        
        ## SweeTile
        # implement algorithm 1
        # construct the configuration
        # 
        
        _file = open(f"{result_dir}/bandwidth_trace_{video}_{user}.pkl", "wb")
        pickle.dump(Bandwidth_trace, _file)
        _file = open(f"{result_dir}/bandwidth_pred_{video}_{user}.pkl", "wb")
        pickle.dump(Bandwidth_pred, _file)
        _file = open(f"{result_dir}/buffer_{video}_{user}.pkl", "wb")
        pickle.dump(Buffer, _file)
        _file = open(f"{result_dir}/weights_{video}_{user}.pkl", "wb")
        pickle.dump(Weights, _file)
        _file = open(f"{result_dir}/received_rate_{video}_{user}.pkl", "wb")
        pickle.dump(Received_rate, _file)
        _file = open(f"{result_dir}/received_qoe_{video}_{user}.pkl", "wb")
        pickle.dump(Received_qoe, _file)
        _file = open(f"{result_dir}/ground_{video}_{user}.pkl", "wb")
        pickle.dump(ground, _file)
        _file = open(f"{result_dir}/prediction_{video}_{user}.pkl", "wb")
        pickle.dump(prediction, _file)

Except2
[994124, 1460132.8, 1926141.6, 2392150.4, 2858159.2, 3324168, 3300645.6, 3277123.2, 3253600.8, 3230078.4, 3206556, 3460365.6, 3714175.2, 3967984.8, 4221794.4, 4475604, 5120579.2, 5765554.4, 6410529.6, 7055504.8, 7700480, 7573342.4, 7446204.8, 7319067.2, 7191929.6, 7064792, 6629643.2, 6194494.4, 5759345.6, 5324196.8, 4889048, 5353030.4, 5817012.8, 6280995.2, 6744977.6, 7208960, 6796568.8, 6384177.6, 5971786.4, 5559395.2, 5147004, 4718486.4, 4289968.8, 3861451.2, 3432933.6, 3004416, 3927654.4, 4850892.8, 5774131.2, 6697369.6, 7620608, 7605043.2, 7589478.4, 7573913.6, 7558348.8, 7542784, 7250313.6, 6957843.2, 6665372.8, 6372902.4, 6080432, 5842232.0, 5604032.0, 5365832.0, 5127632.0, 4889432, 4601721.6, 4314011.2, 4026300.8, 3738590.4, 3450880, 4289331.2, 5127782.4, 5966233.6, 6804684.8, 7643136, 7171217.6, 6699299.2, 6227380.8, 5755462.4, 5283544, 5345680.8, 5407817.6, 5469954.4, 5532091.2, 5594228, 5390910.4, 5187592.8, 4984275.2, 4780957.6, 4577640, 4392019.2, 4206398.4, 4020777

bandwidth_pred:  [994124, 994124, 994124, 994124, 994124, 994124, 994124, 994124, 994124, 994124]
{0: 47.81268986898964, 1: 47.39490738631727, 2: 45.89573866611312, 3: 46.85492883603119, 4: 45.44208947441943, 5: 43.83877148686095, 6: 46.44074713460598, 7: 45.03133948388685, 8: 43.42943286225699, 9: 41.82842560311634, 10: 45.93195826611162, 11: 44.60472700218145, 12: 43.00371613931138, 13: 41.40320796651753, 14: 39.76816012340231, 15: 45.415517330547495, 16: 44.169799260047434, 17: 42.569411506464064, 18: 40.96925058666535, 19: 39.33442651735177, 20: 37.60125315109396, 21: 44.974542205911284, 22: 43.649942119748246, 23: 42.130014897103784, 24: 40.53011066790212, 25: 38.89545202846588, 26: 37.162394913167006, 27: 35.4983577030497}
ground band:  994124
pred band:  994124
buffer:  2.6627867901628717


/usr/lib/python3/dist-packages/ipykernel_launcher.py:42: RuntimeWarning: divide by zero encountered in log


bandwidth_pred:  [994124.0, 397649.6, 477179.52, 572615.424, 687138.5088000001, 625741.41056, 750889.692672, 901067.6312064001, 1081281.1574476801, 1297537.388937216]
{0: 46.317766811935535, 1: 45.767921504173984, 2: 43.68077783593976, 3: 45.160356690977395, 4: 43.08010574718433, 5: 40.82377288356834, 6: 44.434155447633216, 7: 42.437649661177474, 8: 40.18284959501932, 9: 37.85338922670447, 10: 43.77143682603901, 11: 41.77730910067076, 12: 39.52348312049453, 13: 37.194564273265, 14: 34.753085973871194, 15: 43.10006022259631, 16: 41.10758722373788, 17: 38.85443941170736, 18: 36.52589766973294, 19: 34.08466206301902, 20: 31.706141228443588, 21: 42.42355129470717, 22: 40.4323015377735, 23: 38.1796552507446, 24: 35.8513924381585, 25: 33.410336359899986, 26: 31.031941573438218, 27: 28.643208007280073}
ground band:  1460132.8
pred band:  994124.0
buffer:  2.4970279050695945
bandwidth_pred:  [1227128.4, 736277.0399999999, 883532.4480000001, 1060238.9376, 1073461.92512, 996127.7501439999, 11953

bandwidth_pred:  [3253600.8000000003, 3244191.8400000003, 3237605.568, 3234406.5216, 3235272.14592, 3241015.3751040003, 3889218.4501248, 4667062.14014976, 5600474.568179713, 6720569.481815656]
{0: 48.20154270833815, 1: 48.40122195629093, 2: 48.83344434481014, 3: 48.47614609688122, 4: 48.74871899641915, 5: 48.093115300396356, 6: 48.51624053942661, 7: 48.508626334982, 8: 48.00745676966276, 9: 47.585517821451745, 10: 48.54147876098329, 11: 48.24758261573865, 12: 47.90506336046008, 13: 47.48365418495225, 14: 47.02800074653772, 15: 48.55894039603194, 16: 48.135759492417066, 17: 47.713901086368125, 18: 47.37286043202868, 19: 46.917444552475686, 20: 46.523842179029366, 21: 48.24063803479424, 22: 47.93891212208349, 23: 47.59754204724927, 24: 47.25677375917474, 25: 46.801533473141966, 26: 46.40805451944228, 27: 46.003695135785115}
ground band:  3460365.6
pred band:  3253600.8000000003
buffer:  1.2565759082244123
bandwidth_pred:  [3285544.8, 3287229.1199999996, 3293954.784, 3306730.0608, 3326764

...

bandwidth_pred:  [7589478.4, 7583252.480000001, 7578894.336000001, 7576777.523200001, 7577350.307840002, 7581150.609408001, 9097380.731289601, 10916856.877547521, 13100228.253057025, 15720273.90366843]
{0: 48.20154270833815, 1: 48.37225189126483, 2: 48.83344434481014, 3: 48.43628844439913, 4: 48.903246890688074, 5: 49.08533000490649, 6: 48.47055512540274, 7: 48.94063183613964, 8: 49.12400488475745, 9: 48.596666632957074, 10: 48.49212485051812, 11: 48.964176273745416, 12: 48.88068397782677, 13: 48.50233258885046, 14: 48.31332876357319, 15: 48.50704842334243, 16: 48.98047150692013, 17: 48.78101574537941, 18: 48.47863328088521, 19: 48.20621058032046, 20: 47.985315188201426, 21: 48.51803594246902, 22: 48.99247175279599, 23: 48.676904012081735, 24: 48.45040752314021, 25: 48.17451242366382, 26: 47.950703710173265, 27: 47.78925807854818}
ground band:  7542784
pred band:  7589478.4
buffer:  0.3373020331827544
bandwidth_pred:  [7573913.6, 7567687.68, 7563329.536, 7561212.723199999, 7561785.5078

bandwidth_pred:  [5853086.08, 5749122.816, 5682860.9792, 5650986.77504, 5660377.730048, 5719286.876057601, 6863144.25126912, 8235773.101522943, 9882927.721827531, 11859513.266193036]
{0: 48.20154270833815, 1: 48.47064945077088, 2: 48.83344434481014, 3: 48.57275428162431, 4: 48.94294960301186, 5: 48.436646855554294, 6: 48.62765944836348, 7: 48.826643511885585, 8: 48.34224109482882, 9: 48.170541935424495, 10: 48.66231751503287, 11: 48.39926444839257, 12: 48.30334362049882, 13: 48.12501518104169, 14: 47.907114691285585, 15: 48.603358235598314, 16: 48.270095593571384, 17: 48.166255563456836, 18: 47.98352561248933, 19: 47.84393820859293, 20: 47.68909701203223, 21: 48.312996814666526, 22: 48.21196264719452, 23: 48.10145594830629, 24: 47.9130194819235, 25: 47.7736230687933, 26: 47.61891566808562, 27: 47.45509085704297}
ground band:  5127632.0
pred band:  5853086.08
buffer:  0.1779649237604537
bandwidth_pred:  [5604032.0, 5508752.0, 5442056.0, 5409660.8, 5418426.5600000005, 5476585.472, 657190

bandwidth_pred:  [4514563.5200000005, 4669758.144, 4913533.7728, 5038374.28736, 5020492.664832001, 4831344.4777984, 5797613.37335808, 6957136.048029696, 8348563.257635635, 10018275.909162764]
{0: 48.20154270833815, 1: 48.458128689181756, 2: 48.75321438205903, 3: 48.55538783446942, 4: 48.37213157579492, 5: 48.13673735936722, 6: 48.607665996104984, 7: 48.33649275630923, 8: 48.01341619897099, 9: 47.75434104181186, 10: 48.171511060150614, 11: 48.11825950009554, 12: 47.94556852298932, 13: 47.68231595831982, 14: 47.46275885249536, 15: 48.110720453747675, 16: 48.04419981896962, 17: 47.866529663790125, 18: 47.59893000195432, 19: 47.37962414517366, 20: 47.14453517058455, 21: 47.87794405739702, 22: 47.96367705877963, 23: 47.69792029559698, 24: 47.5087959040264, 25: 47.28967613392652, 26: 47.05471758806275, 27: 46.81032301293783}
ground band:  6804684.8
pred band:  4514563.5200000005
buffer:  0.37098837959233144
bandwidth_pred:  [5127782.4, 5463162.88, 5697929.216, 5811958.5792, 5781103.57504, 55

bandwidth_pred:  [5452491.839999999, 5391897.728, 5413568.4736, 5427146.00832, 5431011.689983999, 5423223.1479808, 6507867.777576961, 7809441.333092352, 9371329.599710822, 11245595.519652987]
{0: 48.20154270833815, 1: 48.42256198112499, 2: 48.83344434481014, 3: 48.505861748476796, 4: 48.39982119688825, 5: 48.197151603719774, 6: 48.55052508568101, 7: 48.315469049988856, 8: 48.18119857665915, 9: 47.94795277485241, 10: 48.57867104463519, 11: 48.286177507550626, 12: 48.146802363622754, 13: 47.8288266364307, 14: 47.622239789977755, 15: 48.10060390931795, 16: 48.247504648686224, 17: 48.01914601334781, 18: 47.775843305167456, 19: 47.569499151394396, 20: 47.34586976975984, 21: 48.06243850739338, 22: 48.11816538866913, 23: 47.96426648339005, 24: 47.71702134586183, 25: 47.5108567205442, 26: 47.28735338702329, 27: 47.05351127380602}
ground band:  5532091.2
pred band:  5452491.839999999
buffer:  0.210411437123639
bandwidth_pred:  [5407817.6, 5432672.32, 5450070.624, 5458521.228800001, 5456234.5945

bandwidth_pred:  [4395558.5600000005, 4318478.751999999, 4266646.502400001, 4241571.96288, 4248606.6754560005, 4294172.4905472, 5153006.988656639, 6183608.3863879675, 7420330.063665561, 8904396.076398674]
{0: 48.20154270833815, 1: 48.347096846334736, 2: 48.14931458603691, 3: 48.4015017923603, 4: 48.16025002519923, 5: 47.87121151859706, 6: 48.26296070402194, 7: 48.059003828764304, 8: 47.84150651968514, 9: 47.51043783160417, 10: 48.259449700695825, 11: 48.02270932015918, 12: 47.71691899413025, 13: 47.46356786174219, 14: 47.17171662271862, 15: 48.250298459857376, 16: 47.897063933076296, 17: 47.663225546218136, 18: 47.41016196230531, 19: 47.118496659326915, 20: 46.80635392123875, 21: 47.90370965826888, 22: 47.84452818190942, 23: 47.605737073118334, 24: 47.35288570222001, 25: 47.061357631962785, 26: 46.74931155052415, 27: 46.42500274402298}
ground band:  3835156.8
pred band:  4395558.5600000005
buffer:  0.18844484592230992
2 16_Turtle
Start to process
bandwidth_pred:  [994124, 994124, 99412

In [69]:
record_qoe

{0: 10.449723222864769,
 1: 11.971163505678103,
 2: 13.470451221367998,
 3: 13.082653919432136}

In [64]:
rate_table = np.array([50000., 100000., 200000., 400000., 800000.])
rate_table = rate_table / N_x / N_y

In [65]:
rate_table

array([ 1020.40816327,  2040.81632653,  4081.63265306,  8163.26530612,
       16326.53061224])

In [19]:
record_qoe

{0: array([62.1, 62.1, 62.1, 62.1, 62.1, 62.1, 62.1, 62.1, 62.1, 62.1]),
 1: array([87.34355266, 87.34355266, 87.34355266, 87.34355266, 87.33115354,
        87.33115354, 87.34355266, 87.3747062 , 87.3747062 , 87.33115354]),
 2: array([123.7, 123.7, 123.7, 123.7, 123.7, 123.7, 123.7, 123.7, 123.7,
        123.7]),
 3: array([118.35080237, 118.35080237, 118.35080237, 118.35080237,
        118.33634795, 118.33634795, 118.35080237, 118.38715032,
        118.38715032, 118.33634795]),
 4: array([154.70713095, 154.70713095, 154.70713095, 154.70713095,
        154.70509473, 154.70509473, 154.70713095, 154.71222568,
        154.71222568, 154.70509473]),
 5: array([200.2, 200.2, 200.2, 200.2, 200.2, 200.2, 200.2, 200.2, 200.2,
        200.2]),
 6: array([180.35492621, 180.35492621, 180.35492621, 180.35492621,
        180.33930377, 180.33930377, 180.35492621, 180.39422998,
        180.39422998, 180.33930377]),
 7: array([216.71119027, 216.71119027, 216.71119027, 216.71119027,
        216.70799599

In [23]:
record_rate

{0: array([[[ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0., 10., 10., 10., 10., 10.,  0.],
         [ 0., 10., 10., 10., 10., 10.,  0.],
         [ 0., 10., 10., 10., 10., 10.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.]],
 
        [[ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0., 10., 10., 10., 10., 10.,  0.],
         [ 0., 10., 10., 10., 10., 10.,  0.],
         [ 0., 10., 10., 10., 10., 10.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.]],
 
        [[ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0., 10., 10., 10., 10., 10.,  0.],
         [ 0., 10., 10., 10., 10., 10.,  0.],
         [ 0., 10., 10., 10., 10., 10.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.]],
 
        [[ 0.,  0.,  0

In [31]:
0.03 / 2 / np.pi * 360

1.7188733853924696

In [32]:
a = np.array([ 8.94988358e-01, -1.82004794e-01,  1.47377979e-03])

In [67]:
sum(a * a)

0.834132078021388

In [62]:
a

[0.6589517, -0.0812796056, 0.0398374349]